### Trigger mechanism for Somalia 

IPC trigger design as endorsed early 2020 (not clearly documented but was endorsed as using ML1 forecasts):

- The projected national population in Phase 3 and above exceed 20%, AND 
- (The national population in Phase 3 is projected to increase by 5 percentage points, OR 
- The projected national population in Phase 4 or above is 2.5%)

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from pathlib import Path
import os
import sys
path_mod = f"{Path(os.path.dirname(os.path.abspath(''))).parents[1]}/"
sys.path.append(path_mod)
from indicators.food_insecurity.config import Config
from indicators.food_insecurity.ipc_definemetrics import define_trigger_percentage, define_trigger_increase
from indicators.food_insecurity.utils import compute_percentage_columns

In [2]:
admin_level=0
country="somalia"
#suffix of filenames
suffix=""
config=Config()
parameters = config.parameters(country)
country_folder = os.path.join(config.DIR_PATH, config.ANALYSES_DIR, country)
fewsnet_dir = os.path.join(country_folder, config.DATA_DIR, config.FEWSWORLDPOP_PROCESSED_DIR)
fewsnet_filename = config.FEWSWORLDPOP_PROCESSED_FILENAME.format(country=country,admin_level=admin_level,suffix=suffix)
globalipc_dir=os.path.join(country_folder,config.DATA_DIR, config.GLOBALIPC_PROCESSED_DIR)
globalipc_path=os.path.join(globalipc_dir,f"{country}_globalipc_admin{admin_level}{suffix}.csv")

adm_bound_path= os.path.join(country_folder,config.DATA_DIR,config.SHAPEFILE_DIR,parameters[f"path_admin{admin_level}_shp"])

In [3]:
df_fn=pd.read_csv(os.path.join(fewsnet_dir,fewsnet_filename))
df_fn["source"]="FewsNet"
df_gipc=pd.read_csv(globalipc_path)
df_gipc["source"]="GlobalIPC"

In [4]:
df=pd.concat([df_fn,df_gipc])
df=df.replace(0,np.nan)
df["date"]=pd.to_datetime(df["date"])
df["year"]=df["date"].dt.year
df["month"]=df["date"].dt.month

In [5]:
#the data of 2020-11 is an update and thus doesn't include CS data or projected periods
#add them here manually, where the CS data is set to that of 2020-10
CS_cols=[c for c in df.columns if 'CS' in c]
for c in CS_cols:
    df.loc[df.date=="2020-11-01",c]=df.loc[df.date=="2020-10-01",c].values
df[df.date=="2020-11-01"]=compute_percentage_columns(df[df.date=="2020-11-01"],config)
df.loc[df.date=="2020-11-01","period_ML1"]="Nov - Jan 2021"
df.loc[df.date=="2020-11-01","period_ML2"]="Feb - May 2021"

In [6]:
df["trigger_ML1_3_20"] = df.apply(lambda x: define_trigger_percentage(x,"ML1",3,20),axis=1)
df["trigger_ML1_3_5in"] = df.apply(lambda x: define_trigger_increase(x,"ML1",3,5),axis=1)
df["trigger_ML1_4_2half"] = df.apply(lambda x: define_trigger_percentage(x,"ML1",4,2.5),axis=1)

df["trigger_ML2_3_20"] = df.apply(lambda x: define_trigger_percentage(x,"ML2",3,20),axis=1)
df["trigger_ML2_3_5in"] = df.apply(lambda x: define_trigger_increase(x,"ML2",3,5),axis=1)
df["trigger_ML2_4_2half"] = df.apply(lambda x: define_trigger_percentage(x,"ML2",4,2.5),axis=1)

# determine whether national trigger is met
df['threshold_reached_ML1'] =  np.where((df['trigger_ML1_3_20']==1) & ((df['trigger_ML1_3_5in'] )==1 | (df['trigger_ML1_4_2half'] == 1)), 1, 0)
df['threshold_reached_ML2'] =  np.where((df['trigger_ML2_3_20']==1) & ((df['trigger_ML2_3_5in'] )==1 | (df['trigger_ML2_4_2half'] == 1)), 1, 0)

In [7]:
df[df.date=="2020-11-01"]["pop_CS"]

42    1.580659e+07
Name: pop_CS, dtype: float64

In [8]:
df[df.date=="2020-11-01"][["CS_3p","perc_CS_3p","CS_4p","perc_CS_4p","ML1_3p","perc_ML1_3p","ML1_4p","perc_ML1_4p","ML2_3p","perc_ML2_3p","ML2_4p","perc_ML2_4p"]]

,CS_3p,perc_CS_3p,CS_4p,perc_CS_4p,ML1_3p,perc_ML1_3p,ML1_4p,perc_ML1_4p,ML2_3p,perc_ML2_3p,ML2_4p,perc_ML2_4p
42,3.411650e+06,21.583718,0.0,0.0,3.411650e+06,21.583718,43808.103516,0.277151,5.650173e+06,35.745683,43808.103516,0.277151


FewsNet released an update of projections in November, though they don't differ much from those released in October. 
Combining the two, means our freshest data consists of a "current" assessment in Oct 2020, a short-range projection for Nov 2020-Jan 2021 and a long-range projection for Feb 2021 - May 2021. 
According to our methods, the figures are as follows



| Period        | Population IPC 3+         | Percentage IPC 3+  |Population IPC 4+         | Percentage IPC 4+  |
| ------------- |:-------------------------:| ------------------:|:------------------------:| ------------------:|
| Oct 2020      | 3.4 million               | 21.6%              | 0                        |0                   |
| Nov 2020 - Jan 2021     | 3.4 million     |   21.6%            | 44 thousand             |0.28%               |
| Feb 2021 - May 2021 | 5.6 million      |    35.8% | 44 thousand | 0.28%

There is a large discrepancy between the calculated population in IPC 3+ between our method and that of FewsNet. We don't know how FewsNet's population numbers are computed, however we do know that they assign a phase to an area if 20% of more of the population of that area is in the given phase or above. Since we don't have any information whether this number is between 20% and 100%, with our methodology we assign 100% of the population to the given phase. This might explain the discrepancy.

To compute the percentage of the population in an IPC phase, we use the national population as reported by UNESA, which is 15.8 million for 2020. 

With these numbers, and the current trigger design, we would trigger for the Feb - May 2021 period, as more than 20% of the population is projected to be in IPC3+ and the increase of population in IPC 3+ compared to October is more than 5%. For the Nov 2020 -Jan 2021 the trigger was not met, since the percentage of population projected in IPC3+ didn't increase compared to the October situation, nor was the projected population in IPC 4+ more than 2.5%. 

Regarding the Deyr season of 2016,2017,2018 and 2019, we thought the data released in October and projecting up to May reflects the Deyr season the best. If you think this should instead be the data released in February (covering up to September), please let us know.  

The table below summarizes all the data. As you can see from here, the situation was significantly worse during Oct 2017 - May 2018 compared to now. However, compared to the other 3 years (2016,2017,2019), the sitation of this year (2021) is worse. 

For 2018, the trigger would be met both for the short-range and long-range projections. For 2017 the trigger would have been met for the long-range projections

|Publication date  | Period        | Population IPC 3+         | Percentage IPC 3+  |Population IPC 4+         | Percentage IPC 4+  |
|------------------:| ------------- |:-------------------------:| ------------------:|:------------------------:| ------------------:|
|Oct 2015           | Oct 2015      | 1.4 million               | 10.6%              | 0                        |0%                   |
|Oct 2016           | Oct 2016      | 1.4 million               |10.1%               |0                         |0%                   |  
|Oct 2017           | Oct 2017     | 3.4 million     |   21.6%            | 1.4 million             |9.4%               
|Oct 2018           | Oct 2018 | 1.3 million      |    8.6% | 0 | 0%
| Oct 2020 | Oct 2020      | 3.4 million               | 21.6%              | 0                        |0                   |
|.|  || || |
|Oct 2015           | Oct 2015 - Dec 2015      | 1.5 million               | 11%              | 200 thousand                        |1.5%                   |
|Oct 2016           | Oct 2016 - Jan 2017     |1.4 million |10.1% |0 |0%
|Oct 2017           | Oct 2017 - Jan 2018     | 7.9 million | 54.6% |4.4 million | 30.5%|              
|Oct 2018           | Oct 2018 - Jan 2019 | 700 thousand | 4.6% | 200 thousand | 1.4%|
| Oct 2020 | Nov 2020 - Jan 2021     | 3.4 million     |   21.6%            | 44 thousand             |0.28%               |
|.|  || || |
|Oct 2015           | Jan 2016 - Mar 2016      | 900 thousand | 6.6% | 0 | 0%|
|Oct 2016           | Feb 2017 - May 2017 | 3.2 million | 22.4% | 0 |0%|
|Oct 2017           | Feb 2018 - May 2018     |   10.6 million | 73.4% |6.4 million | 43.9% |            
|Oct 2018           | Feb 2019 - May 2019 | 1.8 million | 12.2% | 200 thousand | 1.4% |
| Oct 2020| Feb 2021 - May 2021 | 5.6 million      |    35.8% | 44 thousand | 0.28%

In [9]:
df_neat_cs=df[["date","CS_3p","perc_CS_3p","CS_4p","perc_CS_4p"]].rename(columns={"date":"Publication date","CS_3p":"Population IPC 3+","perc_CS_3p":"Percentage IPC 3+","CS_4p":"Population IPC 4+","perc_CS_4p":"Percentage IPC 4+"})
df_neat_cs["Period"]=df_neat_cs["Publication date"]

In [10]:
df_neat_cs[df_neat_cs["Publication date"].isin(["2016-02-01","2017-02-01","2018-02-01","2019-02-01"])][['Publication date', 'Period', 'Population IPC 3+', 'Percentage IPC 3+',
       'Population IPC 4+', 'Percentage IPC 4+']].replace(np.nan,0).style.format({"Publication date":"{:%Y-%m}","Period":"{:%Y-%m}",'Population IPC 3+': "{:,.0f}", 'Percentage IPC 3+': '{:.1f}%','Population IPC 4+': "{:,.0f}",'Percentage IPC 4+': '{:.1f}%'})

,Publication date,Period,Population IPC 3+,Percentage IPC 3+,Population IPC 4+,Percentage IPC 4+
26,2016-02,2016-02,"1,678,026",11.9%,0,0.0%
29,2017-02,2017-02,"4,122,092",28.4%,"1,593,350",11.0%
32,2018-02,2018-02,"4,448,669",29.8%,0,0.0%
35,2019-02,2019-02,"396,463",2.6%,0,0.0%


In [11]:
p="ML1"
df_neat_ml1=df[["date",f"period_{p}",f"{p}_3p",f"perc_{p}_3p",f"{p}_4p",f"perc_{p}_4p"]].rename(columns={"date":"Publication date",f"period_{p}":"Period",f"{p}_3p":"Population IPC 3+",f"perc_{p}_3p":"Percentage IPC 3+",f"{p}_4p":"Population IPC 4+",f"perc_{p}_4p":"Percentage IPC 4+"})

In [12]:
df_neat_ml1[df_neat_ml1["Publication date"].isin(["2016-02-01","2017-02-01","2018-02-01","2019-02-01"])][['Publication date', 'Period', 'Population IPC 3+', 'Percentage IPC 3+',
       'Population IPC 4+', 'Percentage IPC 4+']].replace(np.nan,0).style.format({"Publication date":"{:%Y-%m}",'Population IPC 3+': "{:,.0f}", 'Percentage IPC 3+': '{:.1f}%','Population IPC 4+': "{:,.0f}",'Percentage IPC 4+': '{:.1f}%'})

,Publication date,Period,Population IPC 3+,Percentage IPC 3+,Population IPC 4+,Percentage IPC 4+
26,2016-02,Feb - May 2016,"1,678,026",11.9%,0,0.0%
29,2017-02,Feb - May 2017,"7,734,301",53.3%,"1,597,840",11.0%
32,2018-02,Feb - May 2018,"7,883,964",52.8%,"1,136,212",7.6%
35,2019-02,Feb - May 2019,"4,500,705",29.3%,0,0.0%


In [13]:
p="ML2"
df_neat_ml2=df[["date",f"period_{p}",f"{p}_3p",f"perc_{p}_3p",f"{p}_4p",f"perc_{p}_4p"]].rename(columns={"date":"Publication date",f"period_{p}":"Period",f"{p}_3p":"Population IPC 3+",f"perc_{p}_3p":"Percentage IPC 3+",f"{p}_4p":"Population IPC 4+",f"perc_{p}_4p":"Percentage IPC 4+"})

In [14]:
df_neat_ml2[df_neat_ml2["Publication date"].isin(["2016-02-01","2017-02-01","2018-02-01","2019-02-01"])][['Publication date', 'Period', 'Population IPC 3+', 'Percentage IPC 3+',
       'Population IPC 4+', 'Percentage IPC 4+']].replace(np.nan,0).style.format({"Publication date":"{:%Y-%m}",'Population IPC 3+': "{:,.0f}", 'Percentage IPC 3+': '{:.1f}%','Population IPC 4+': "{:,.0f}",'Percentage IPC 4+': '{:.1f}%'})

,Publication date,Period,Population IPC 3+,Percentage IPC 3+,Population IPC 4+,Percentage IPC 4+
26,2016-02,Jun - Sep 2016,"184,872",1.3%,0,0.0%
29,2017-02,Jun - Sep 2017,"8,019,696",55.3%,"2,223,490",15.3%
32,2018-02,Jun - Sep 2018,"6,721,830",45.0%,"213,425",1.4%
35,2019-02,Jun - Sep 2019,"220,915",1.4%,"220,915",1.4%


In [15]:
df[df.date.isin(["2015-10-01","2016-10-01","2017-10-01","2018-10-01","2019-10-01","2020-11-01"])][["date","period_ML1","period_ML2","CS_3p","perc_CS_3p","CS_4p","perc_CS_4p","ML1_3p","perc_ML1_3p","ML1_4p","perc_ML1_4p","ML2_3p","perc_ML2_3p","ML2_4p","perc_ML2_4p"]]

,date,period_ML1,period_ML2,CS_3p,perc_CS_3p,CS_4p,perc_CS_4p,ML1_3p,perc_ML1_3p,ML1_4p,perc_ML1_4p,ML2_3p,perc_ML2_3p,ML2_4p,perc_ML2_4p
25,2015-10-01,Oct - Dec 2015,Jan - Mar 2016,1.452450e+06,10.583108,NaN,NaN,1.512830e+06,11.023064,2.086846e+05,1.520557,9.033625e+05,6.582248,NaN,NaN
28,2016-10-01,Oct - Jan 2017,Feb - May 2017,1.425895e+06,10.102733,NaN,NaN,1.425895e+06,10.102733,NaN,NaN,3.161653e+06,22.400906,NaN,NaN
31,2017-10-01,Oct - Jan 2018,Feb - May 2018,8.645656e+06,59.573980,1.362915e+06,9.391336,7.916773e+06,54.551518,4.425818e+06,30.496655,1.065692e+07,73.432840,6.374538e+06,43.924557
34,2018-10-01,Oct - Jan 2019,Feb - May 2019,1.280308e+06,8.576946,NaN,NaN,6.856531e+05,4.593278,2.134249e+05,1.429760,1.816148e+06,12.166611,2.134249e+05,1.429760
37,2019-10-01,Oct - Jan 2020,Feb - May 2020,5.393036e+06,35.114616,6.307950e+05,4.107171,9.143139e+06,59.531923,8.520734e+05,5.547938,9.107228e+05,5.929810,NaN,NaN
42,2020-11-01,Nov - Jan 2021,Feb - May 2021,3.411650e+06,21.583718,0.000000e+00,0.000000,3.411650e+06,21.583718,4.380810e+04,0.277151,5.650173e+06,35.745683,4.380810e+04,0.277151


In [20]:
df[df.threshold_reached_ML1==1][["date","perc_ML1_3p","perc_ML1_4p","perc_CS_3p","perc_inc_ML1_3p","source"]]

,date,perc_ML1_3p,perc_ML1_4p,perc_CS_3p,perc_inc_ML1_3p,source
5,2010-10-01,55.212803,30.851796,47.051393,8.161410,FewsNet
6,2011-01-01,60.233880,32.042486,46.961206,13.272673,FewsNet
7,2011-04-01,78.453241,33.225925,47.467930,30.985311,FewsNet
10,2012-01-01,42.998485,9.357919,33.074492,9.923993,FewsNet
29,2017-02-01,53.294169,11.010120,28.403790,24.890379,FewsNet
30,2017-06-01,62.765370,30.562141,51.378078,11.387292,FewsNet
32,2018-02-01,52.815687,7.611627,29.802206,23.013481,FewsNet
33,2018-06-01,27.107145,1.429760,15.266251,11.840894,FewsNet
35,2019-02-01,29.304553,NaN,2.581409,26.723145,FewsNet
36,2019-06-01,51.468845,17.883884,37.950998,13.517847,FewsNet


In [19]:
df[df.threshold_reached_ML2==1][["date","perc_ML2_3p","perc_ML2_4p","perc_CS_3p","perc_inc_ML2_3p","source"]]

,date,perc_ML2_3p,perc_ML2_4p,perc_CS_3p,perc_inc_ML2_3p,source
2,2010-01-01,80.054980,38.637093,74.384857,5.670123,FewsNet
4,2010-07-01,70.318834,23.053684,43.219358,27.099476,FewsNet
5,2010-10-01,55.212953,33.858614,47.051393,8.161561,FewsNet
7,2011-04-01,84.887623,37.421485,47.467930,37.419694,FewsNet
10,2012-01-01,42.998485,9.357919,33.074492,9.923993,FewsNet
27,2016-06-01,47.477225,NaN,12.736185,34.741040,FewsNet
28,2016-10-01,22.400906,NaN,10.102733,12.298173,FewsNet
29,2017-02-01,55.260721,15.321237,28.403790,26.856931,FewsNet
30,2017-06-01,62.723858,18.849667,51.378078,11.345780,FewsNet
31,2017-10-01,73.432840,43.924557,59.573980,13.858860,FewsNet


In [22]:
df_gipc=df[df.source=="GlobalIPC"]

In [25]:
df_gipc[df_gipc.date.isin(["2017-01","2017-07","2018-01"])][["date","perc_ML1_3p","perc_CS_3p"]]

,date,perc_ML1_3p,perc_CS_3p
0,2017-01-01,19.477616,9.790518
1,2017-07-01,12.056432,21.799246
2,2018-01-01,18.184288,11.012483
